<a href="https://colab.research.google.com/github/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/TA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Datos

In [ ]:
import tarfile
import numpy as np
import pandas as pd

## Recopilación

In [ ]:
!wget -P /content/Data https://github.com/renato145/3D-ORGAN/blob/master/datasets/arq_dataset.tar.gz?raw=true

--2022-09-25 06:03:10--  https://github.com/renato145/3D-ORGAN/blob/master/datasets/arq_dataset.tar.gz?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/renato145/3D-ORGAN/raw/master/datasets/arq_dataset.tar.gz [following]
--2022-09-25 06:03:10--  https://github.com/renato145/3D-ORGAN/raw/master/datasets/arq_dataset.tar.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/renato145/3D-ORGAN/master/datasets/arq_dataset.tar.gz [following]
--2022-09-25 06:03:10--  https://raw.githubusercontent.com/renato145/3D-ORGAN/master/datasets/arq_dataset.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199

In [ ]:
folder = "/content/Data/"
file_tar = folder + "arq_dataset.tar.gz?raw=true"
file_name = "custom_arq_dataset.npy"

In [ ]:
file = tarfile.open(file_tar)
file.extract(file_name, folder)
file.close()

In [ ]:
data = np.load(folder+file_name, allow_pickle=True)
data = data.reshape(1)

In [ ]:
data

array([{'train': {'labels': ['bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'ba

## A dataframe

In [ ]:
train = data[0]["train"] #'labels' 'data'
train = [train["labels"],train["data"]]
#train
train[0][-1], train[1][0]

('arq', array([[[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],
 
        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],
 
        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, 

In [ ]:
df = pd.DataFrame(train)
df = df.T
df.columns = ['labels','data']
df

,labels,data
0,bathtub,"[[[False, False, False, False, False, False, F..."
1,bathtub,"[[[False, False, False, False, False, False, F..."
2,bathtub,"[[[False, False, False, False, False, False, F..."
3,bathtub,"[[[False, False, False, False, False, False, F..."
4,bathtub,"[[[False, False, False, False, False, False, F..."
...,...,...
4918,arq,"[[[False, False, False, False, False, False, F..."
4919,arq,"[[[False, False, False, False, False, False, F..."
4920,arq,"[[[False, False, False, False, False, False, F..."
4921,arq,"[[[False, False, False, False, False, False, F..."


## Selección de tipo de objeto

In [ ]:
tipos = df['labels'].unique()
tipos

array(['bathtub', 'dresser', 'chair', 'night_stand', 'toilet', 'monitor',
       'table', 'bed', 'sofa', 'desk', 'arq'], dtype=object)

In [ ]:
tipo = "sofa"  #@param ['bathtub', 'dresser', 'chair', 'night_stand', 'toilet', 'monitor', 'table', 'bed', 'sofa', 'desk', 'arq']
data_tipo = df[ df['labels']==tipo ]
data_tipo

,labels,data
3111,sofa,"[[[False, False, False, False, False, False, F..."
3112,sofa,"[[[False, False, False, False, False, False, F..."
3113,sofa,"[[[False, False, False, False, False, False, F..."
3114,sofa,"[[[False, False, False, False, False, False, F..."
3115,sofa,"[[[False, False, False, False, False, False, F..."
...,...,...
3786,sofa,"[[[False, False, False, False, False, False, F..."
3787,sofa,"[[[False, False, False, False, False, False, F..."
3788,sofa,"[[[False, False, False, False, False, False, F..."
3789,sofa,"[[[False, False, False, False, False, False, F..."


# Conversión de Binvox a STL